In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics

from zoo.zouwu.preprocessing.impute.LastFill import LastFill,MeanFill
from zoo.zouwu.preprocessing.impute.LastFill import MFFill,KNNFill

## Prepare data 

In [2]:
raw_df = pd.read_csv("data/data.csv")

In [3]:
df = pd.DataFrame(pd.to_datetime(raw_df.StartTime))

In [4]:
raw_df.AvgRate.str[-4:].unique()
# Unify AvgRate value
df['AvgRate'] = raw_df.AvgRate.apply(lambda x: float(x[:-4]) if x.endswith("Mbps") else float(x[:-4]) * 1000)
df["total"] = raw_df["total"]
df.set_index("StartTime", inplace=True)

In [5]:
df.head()

,AvgRate,total
StartTime,,
2018-01-01 00:00:00,306.23,275605455598
2018-01-01 02:00:00,285.03,256527692256
2018-01-01 04:00:00,247.39,222652190823
2018-01-01 06:00:00,211.55,190396029658
2018-01-01 08:00:00,234.82,211340468977


choose a part of raw data to do evaluate

In [6]:
sampled_df = df["2019-01-01 00:00:00":"2019-06-30 23:00:00"]

In [7]:
full_idx = pd.date_range(start=sampled_df.index.min(), end=sampled_df.index.max(), freq='2H')
sampled_df = df.reindex(full_idx)
print("no. of n/a values:")
print(sampled_df.isna().sum())

no. of n/a values:
AvgRate    0
total      0
dtype: int64


In [8]:
sampled_df

,AvgRate,total
2019-01-01 00:00:00,271.53,244373500495
2019-01-01 02:00:00,208.49,187638154842
2019-01-01 04:00:00,175.88,158293382443
2019-01-01 06:00:00,229.20,206284366389
2019-01-01 08:00:00,264.95,238459494810
...,...,...
2019-06-30 14:00:00,618.90,557013671978
2019-06-30 16:00:00,533.31,479975450009
2019-06-30 18:00:00,475.86,428273046817
2019-06-30 20:00:00,448.12,403312442571


## Data Imputation with Last Seen Value

use LastFill method to fill missing values for all data

In [9]:
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='2H')
df = df.reindex(full_idx)
print("no. of n/a values:")
print(df.isna().sum())

no. of n/a values:
AvgRate    3
total      3
dtype: int64


In [10]:
last_fill = LastFill()

In [11]:
lastfilled_df = last_fill.impute(df)

In [12]:
print("no. of n/a values:")
print(lastfilled_df.isna().sum())

no. of n/a values:
AvgRate    0
total      0
dtype: int64


In [13]:
lastfilled_df

,AvgRate,total
2018-01-01 00:00:00,306.23,2.756055e+11
2018-01-01 02:00:00,285.03,2.565277e+11
2018-01-01 04:00:00,247.39,2.226522e+11
2018-01-01 06:00:00,211.55,1.903960e+11
2018-01-01 08:00:00,234.82,2.113405e+11
...,...,...
2019-12-31 14:00:00,255.08,2.295759e+11
2019-12-31 16:00:00,192.44,1.731922e+11
2019-12-31 18:00:00,209.65,1.886873e+11
2019-12-31 20:00:00,211.22,1.900964e+11


evaluate LastFill method with sample data

In [14]:
mse_10_lf = last_fill.evaluate(sampled_df, 0.1)
mse_10_lf

[2155.109979189687, 2.2178545481568145e+21]

In [15]:
mse_50_last = last_fill.evaluate(sampled_df, 0.5)
mse_50_last

[27494.135862292816, 2.5725095039822643e+22]

## Data Imputation with Mean Value

In [16]:
print("no. of n/a values:")
print(df.isna().sum())

no. of n/a values:
AvgRate    3
total      3
dtype: int64


In [17]:
mean_fill = MeanFill()
meanfilled_df = mean_fill.impute(df)

In [18]:
print("no. of n/a values:")
print(meanfilled_df.isna().sum())

no. of n/a values:
AvgRate    0
total      0
dtype: int64


In [19]:
meanfilled_df 

,AvgRate,total
2018-01-01 00:00:00,306.23,2.756055e+11
2018-01-01 02:00:00,285.03,2.565277e+11
2018-01-01 04:00:00,247.39,2.226522e+11
2018-01-01 06:00:00,211.55,1.903960e+11
2018-01-01 08:00:00,234.82,2.113405e+11
...,...,...
2019-12-31 14:00:00,255.08,2.295759e+11
2019-12-31 16:00:00,192.44,1.731922e+11
2019-12-31 18:00:00,209.65,1.886873e+11
2019-12-31 20:00:00,211.22,1.900964e+11


evaluate MeanFill method with sample data

In [20]:
mse_10_mean = mean_fill.evaluate(sampled_df, 0.1)
mse_10_mean

[3614.615450604725, 1.8795196425198331e+21]

In [21]:
mse_50_mean = mean_fill.evaluate(sampled_df, 0.5)
mse_50_mean

[13603.993516502194, 9.86914052313247e+21]

## Data Imputation with Matrix Factorization

In [22]:
print("no. of n/a values:")
print(df.isna().sum())

no. of n/a values:
AvgRate    3
total      3
dtype: int64


In [23]:
mf_fill = MFFill()

In [24]:
mffilled_df = mf_fill.impute(df)

Iteration: 20 ; error = 290.4531
Iteration: 40 ; error = 68.2078
Iteration: 60 ; error = 21.7960
Iteration: 80 ; error = 9.7185
Iteration: 100 ; error = 6.1996


In [25]:
print("no. of n/a values:")
print(mffilled_df.isna().sum())

no. of n/a values:
AvgRate    0
total      0
dtype: int64


In [26]:
mffilled_df

,AvgRate,total
2018-01-01 00:00:00,306.23,2.756055e+11
2018-01-01 02:00:00,285.03,2.565277e+11
2018-01-01 04:00:00,247.39,2.226522e+11
2018-01-01 06:00:00,211.55,1.903960e+11
2018-01-01 08:00:00,234.82,2.113405e+11
...,...,...
2019-12-31 14:00:00,255.08,2.295759e+11
2019-12-31 16:00:00,192.44,1.731922e+11
2019-12-31 18:00:00,209.65,1.886873e+11
2019-12-31 20:00:00,211.22,1.900964e+11


evaluate MF method with sample data

In [27]:
mse_10_mf = mf_fill.evaluate(sampled_df, 0.1)
mse_10_mf

Iteration: 20 ; error = 58.9418
Iteration: 40 ; error = 26.3166
Iteration: 60 ; error = 12.2453
Iteration: 80 ; error = 6.1744
Iteration: 100 ; error = 3.3287
Iteration: 20 ; error = 53.5261
Iteration: 40 ; error = 23.8888
Iteration: 60 ; error = 11.6441
Iteration: 80 ; error = 5.7063
Iteration: 100 ; error = 3.0592


[1030.0484793305225, 7.47098866576721e+20]

In [28]:
mse_50_mf = mf_fill.evaluate(sampled_df, 0.5)
mse_50_mf

Iteration: 20 ; error = 53.7176
Iteration: 40 ; error = 23.7342
Iteration: 60 ; error = 11.3311
Iteration: 80 ; error = 5.7237
Iteration: 100 ; error = 3.0257
Iteration: 20 ; error = 31.1758
Iteration: 40 ; error = 14.2583
Iteration: 60 ; error = 6.6276
Iteration: 80 ; error = 3.3382
Iteration: 100 ; error = 1.7775


[4511.392369182612, 4.413616041496677e+21]

## Data Imputation with KNN

In [29]:
knn_fill = KNNFill()

evaluate KNN method with sample data

In [32]:
mse_10_knn = knn_fill.evaluate(sampled_df, 0.1,knn=True)
mse_10_knn

[1111.2744005819454, 4.3434029011095645e+21]

In [31]:
mse_50_knn = knn_fill.evaluate(sampled_df, 0.5,knn=True)
mse_50_knn

[15313.438057961108, 1.42781828508412e+22]